From the Exploratory Data Analysis Notebook, we see that skewness is negligible. Hence no transformation like Log or Boxcox is necessary.
Also There are no missing values & No Categorical Varibales present.

Also we did exhaustive parameter tuning for lasso, ridge , XGBoost, GradientBoosting, LightGBM, in the first notebook. Therefore I am directly using the models with best results so far in this notebook and then do further parameter tuning by comparing the RMSE through a 5 fold Cross Validation.

To check in detail, how I did parameter Tuning for each model, please refer the "Exploratory Data Analysis" Notebook

In [1]:
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew 
from scipy.stats import shapiro #for checking for skewed data - Shapiro Wilks Test
import matplotlib
import seaborn as sns

In [3]:
import pandas as pd
import numpy as np

In [46]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [33]:
uni =pd.read_csv('UNI.csv')
uni.head()
# loading the training data

,v.id,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque,current price
0,1,535651,798186,3,78945,1,2,14,177,73,123,351318.0
1,2,591911,861056,6,117220,5,9,9,148,74,95,285001.5
2,3,686990,770762,2,132538,2,8,15,181,53,97,215386.0
3,4,573999,722381,4,101065,4,3,11,197,54,116,244295.5
4,5,691388,811335,6,61559,3,9,12,160,53,105,531114.5


In [34]:
unitest = pd.read_csv('unitest_1.csv')
unitest.head()
#loading the test data

,v.id,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque
0,1,514396,881378,3,115717,3,4,10,144,70,117
1,2,654594,749943,7,135999,1,2,13,161,51,70
2,3,559836,865594,4,142651,1,6,10,184,116,72
3,4,523784,870895,5,57223,4,2,14,170,76,102
4,5,547534,824478,4,51927,5,6,9,162,53,71


In [35]:
ids = unitest['v.id']


In [36]:
uni.drop(['v.id'], axis =1, inplace =True)

unitest.drop(['v.id'], axis =1, inplace =True)
# we are dropping this column as its value does not play any role in price prediction. it is just a serial no.

In [37]:
target = uni['current price']


In [38]:
uni.shape

(1000, 11)

In [41]:
uni.drop(['current price'], axis =1, inplace =True)

uni.head()
# dropping the target i.e. current price from training data 

,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque
0,535651,798186,3,78945,1,2,14,177,73,123
1,591911,861056,6,117220,5,9,9,148,74,95
2,686990,770762,2,132538,2,8,15,181,53,97
3,573999,722381,4,101065,4,3,11,197,54,116
4,691388,811335,6,61559,3,9,12,160,53,105


In [40]:
unitest.head()

,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque
0,514396,881378,3,115717,3,4,10,144,70,117
1,654594,749943,7,135999,1,2,13,161,51,70
2,559836,865594,4,142651,1,6,10,184,116,72
3,523784,870895,5,57223,4,2,14,170,76,102
4,547534,824478,4,51927,5,6,9,162,53,71


In [42]:

#Centering the data around mean 0 and variance 1 as it helps many algos to perform better
#standardizing data
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
 # Don't cheat - fit only on training data
scaler.fit(uni)  
uni = scaler.transform(uni)  
# apply same transformation to test data
unitest = scaler.transform(unitest)  

In [43]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(uni)
    rmse= np.sqrt(-cross_val_score(model, uni, target,scoring="neg_mean_squared_error", cv=5))
    return(rmse)

In [129]:
KRR2 = KernelRidge(alpha=0.4, kernel='polynomial', degree=2, coef0=1.5)
score = rmsle_cv(KRR2)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))


 KRRt score: 3048.6715662182 (134.7469878999)



In [130]:
KRR2.fit(uni, target)

KernelRidge(alpha=0.4, coef0=1.5, degree=2, gamma=None, kernel='polynomial',
            kernel_params=None)

In [131]:
pred = KRR2.predict(unitest)

In [50]:
res = pd.DataFrame()
res['v.id']= ids
res['current price'] = pred
res.to_csv('submissionK.csv',index=False)

In [190]:
#We already did exhaustive Parameter Tuning in the First Notebook. Loading the best Model so far here, directly
# After increasing feature fraction , performance improved significantly - which is expected -
#as we have such a small no of features
model_lgb3 = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=9200,
                              max_bin = 55, bagging_fraction = 0.65,
                              bagging_freq = 5, feature_fraction = 0.8,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb3)
print("LGBM score: {:.10f} ({:.10f})\n" .format(score.mean(), score.std()))

LGBM score: 5609.7203723480 (143.2407188594)



In [188]:
#increasing the feature fraction from 0.5 in Notebook 1 to 0.7
model_lgb4 = lgb.LGBMRegressor(objective='regression',num_leaves=5, max_depth=4,
                              learning_rate=0.05, n_estimators=9200,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.7,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb4)
print("LGBM score: {:.10f} ({:.10f})\n" .format(score.mean(), score.std()))

LGBM score: 5266.0718203960 (158.8659268561)



In [55]:
#As explained in Notebook 1, RobustScaler is used as Lasso & Enet are sensitive to outliers
lasso455 = make_pipeline(RobustScaler(), Lasso(alpha =0.00001, random_state=1))
score = rmsle_cv(lasso455)
print("\nLasso score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))


Lasso score: 8808.5091692567 (214.4243425916)



In [106]:
lasso455 = make_pipeline(RobustScaler(), Lasso(alpha =5, random_state=1, normalize = True))
score = rmsle_cv(lasso455)
print("\nLasso score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))


Lasso score: 8801.8238165973 (219.9348127326)



In [63]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("\nElastic Net score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))


Elastic Net score: 8808.4891565338 (214.3812572783)



In [127]:
# performance improved significantly as I changed the loss from 'huber' to 'ls' as our evaluation metric is RMSE
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,loss ='ls',
                                   max_depth=3, 
                                   min_samples_leaf=15, min_samples_split=10, 
                                    random_state =5)

In [92]:
scoregb = rmsle_cv(GBoost)
print("\n GBoost score: {:.20f} ({:.10f})\n".format(scoregb.mean(), scoregb.std()))



 GBoost score: 7056.55954901463519490790 (124.3584756462)



In [70]:
GBoost2 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=10, min_samples_split=10, 
                                   loss='huber', random_state =5)
scoregb2 = rmsle_cv(GBoost2)
print("\n GBoost score: {:.20f} ({:.10f})\n".format(scoregb2.mean(), scoregb.std()))
# error reduced almost to half. Significant Training time


 GBoost score: 9998.52111499206148437224 (208.3872393601)



In [71]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,eval_metric  ='rmse',
                             random_state =7, nthread = -1)
scorexgb = rmsle_cv(model_xgb)
print("\n XGBoost score: {:.30f} ({:.10f})\n".format(scorexgb.mean(), scorexgb.std()))



 XGBoost score: 7082.491658019792339473497122526169 (404.9964506365)



In [125]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.9603, gamma=5, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.7213, silent=1,eval_metric  ='rmse',
                             random_state =7, nthread = -1)
scorexgb = rmsle_cv(model_xgb)
print("\n XGBoost score: {:.30f} ({:.10f})\n".format(scorexgb.mean(), scorexgb.std()))


 XGBoost score: 6193.901503241211685235612094402313 (349.5596124231)



In [107]:
#CReating an Ensemble Average Model 
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [108]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR2, lasso455))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 5368.7250 (296.1797)



In [111]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [113]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost,lasso455 ),
                                                 meta_model = KRR2 )

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [114]:
averaged_models.fit(uni,target)

AveragingModels(models=(Pipeline(memory=None,
                                 steps=[('robustscaler',
                                         RobustScaler(copy=True,
                                                      quantile_range=(25.0,
                                                                      75.0),
                                                      with_centering=True,
                                                      with_scaling=True)),
                                        ('elasticnet',
                                         ElasticNet(alpha=0.0005, copy_X=True,
                                                    fit_intercept=True,
                                                    l1_ratio=0.9, max_iter=1000,
                                                    normalize=False,
                                                    positive=False,
                                                    precompute=False,
                                   

In [116]:
predAv = averaged_models.predict(unitest)

In [182]:
 model_lgb3.fit(uni,target)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.5, feature_fraction_seed=9,
              importance_type='split', learning_rate=0.05, max_bin=55,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=6, min_split_gain=0.0,
              min_sum_hessian_in_leaf=11, n_estimators=9200, n_jobs=-1,
              num_leaves=5, objective='regression', random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [184]:
predlg =model_lgb3.predict(unitest)

In [185]:
res = pd.DataFrame()
res['v.id']= ids
res['current price'] = predlg
res.to_csv('submissionLG.csv',index=False)

In [126]:
model_xgb.fit(uni, target)
predxg = model_xgb.predict(unitest)

In [128]:
GBoost.fit(uni, target)
predGB = GBoost.predict(unitest)

In [133]:
ensemblepred = pred *0.3 +predAv*0.25 + predlg*0.17+ predxg*0.15 + predGB*0.8

In [134]:
res = pd.DataFrame()
res['v.id']= ids
res['current price'] = ensemblepred
res.to_csv('submissionEnsemble.csv',index=False)
#THIS MODEL led to overfitting & fetched low scores 

In [135]:
from sklearn.neural_network import MLPRegressor


In [144]:
reg = MLPRegressor(hidden_layer_sizes=(10, ), activation='relu', solver='adam', alpha=0.005, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.09, power_t=0.5, max_iter=1200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)

In [145]:
score = rmsle_cv(reg)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))
# Multi Layer Perceptron giving a horrible performance despite parameter Tuning


 KRRt score: 27844.5237887432 (4277.2242636227)



In [162]:
 from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(uni, target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

In [163]:
score = rmsle_cv(neigh)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))
#KNN also giving horrible performance


 KRRt score: 53178.8612265036 (2283.3045303289)



In [178]:
from sklearn.svm import SVR
clf = SVR(gamma=10, C=100, epsilon=0.2)
clf.fit(uni, target)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma=10,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [179]:
score = rmsle_cv(clf)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))
#SVR Performance is horrible too


 KRRt score: 126115.0961467307 (5110.4384450973)



In [215]:
# changing alpha didn't have much affect
KRR3 = KernelRidge(alpha=0.001, kernel='polynomial', degree=2, coef0=1.5)
score = rmsle_cv(KRR3)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))


 KRRt score: 3001.0326227241 (119.0096640446)



In [216]:
KRR3.fit(uni, target)

KernelRidge(alpha=0.001, coef0=1.5, degree=2, gamma=None, kernel='polynomial',
            kernel_params=None)

In [218]:
pred = KRR3.predict(unitest)

In [219]:
res = pd.DataFrame()
res['v.id']= ids
res['current price'] = pred
res.to_csv('submissionK2.csv',index=False)

Mean Score vs coeff0

8808.5109198283 1.5
725.5129625918 1.8
725.5091417612 2
725.5033558347 2.5
725.5012414550 2.8
725.5001493636 3
725.4992388077 3.2
725.4981263355 3.5
725.4975132587  3.7
725.4967384590 4

 725.4957290072 
 725.4949627620 5
  725.4938775703 6
  725.4931470025 7
  725.4926213838 8
  725.4919164365  10
  725.4910316235  15
  725.4903651954  25
  725.4900770170  35
  725.4899842259  45
  725.4897321013 60
  725.4898662731  62
  725.4895798132  65----------------Lowest
  725.4898316632 70
  725.4896295919  75
  725.4898739979  85
  725.4899161936 95

In [262]:
#As I increase the degree of polynomial from 2 to 3 the score improves significantly from 3048 to 725.4895798132 
KRR3 = KernelRidge(alpha=0.001, kernel='polynomial', degree=3, coef0=65)
score = rmsle_cv(KRR3)
print("\n KRRt score: {:.10f} ({:.10f})\n".format(score.mean(), score.std()))



 KRRt score: 725.4895798132 (20.3894764462)



In [264]:
KRR3.fit(uni, target)
pred = KRR3.predict(unitest)

In [265]:
res = pd.DataFrame()
res['v.id']= ids
res['current price'] = pred
res.to_csv('submissionKFinal.csv',index=False)